In [1]:
import warnings
from importlib import reload
from functools import partial, lru_cache

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
from joblib import delayed
from modAL import batch
from art.metrics import empirical_robustness
from art.attacks.evasion import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate
from art.estimators.classification.scikitlearn import ScikitlearnSVC
from sklearn.metrics.pairwise import paired_distances, euclidean_distances
import scipy

from ipynb.fs.defs import Bias
from ipynb.fs.defs.Datasets import generateData_twoPills_2D, generateData_twoPills_noNoise_2D, plot_dataset_2D

import libactive
import libadversarial
from libactive import MyActiveLearner, active_split
from libadversarial import adversarial, uncertainty, random_batch
from libutil import ProgressParallel
from libdatasets import *
import librun
from librun import run

Using sklearn


In [2]:
def xpartial(func, *args, **kwargs):
    wrap = partial(func, *args, **kwargs)
    wrap.is_adversarial = func in [adversarial]
    return wrap

In [3]:
matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        ("car", lambda: lru_cache()(car)()),
        ("german", lambda: lru_cache()(german)()),
        ("mnist", lambda: lru_cache()(mnist)()),
    ],
    "dataset_mutators": {
        "none": (lambda *x: x),
    },
    "methods": [
        ("random", xpartial(random_batch, n_instances=10)),
        ("uncertainty", xpartial(batch.uncertainty_batch_sampling, n_instances=10)),
        ("deepfool", xpartial(adversarial, Attack=partial(DeepFool, verbose=False), n_instances=10)),
        ("deepfool w/ adversarial", xpartial(adversarial, Attack=partial(DeepFool, verbose=False), n_instances=10, teach_adversarial=True)),
    ],
    "models": [
        "svm-linear"
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 0.1,
        "n_runs": 10
    }
}

In [4]:
results = librun.run(matrix)

KeyboardInterrupt: 

In [ ]:
print('a')